# Write to actuators

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
%matplotlib qt
import sys
sys.path.insert(0, 'C:\EnergyPlusV23-1-0')  # add E-Plus directory to path to be able to import API
from pyenergyplus.api import EnergyPlusAPI #import EnergyPlus library

In [4]:
# initializations
heatingSch_hndl = 0
coolingSch_hndl = 0
heatingSP_hndl = 0
coolingSP_hndl = 0
outdoorT_hndl = 0
indoorT_hndl = 0
heat_coil_hndl = 0
handleDone = False

In [5]:
def my_callback_function(state):
    # global variables are necessary as the callback function takes only one input: state
    global heatingSch_hndl, coolingSch_hndl,heatingSP_hndl,coolingSP_hndl,indoorT_hndl,outdoorT_hndl,heat_coil_hndl, handleDone
    # get handles
    if not handleDone:
        if api.exchange.api_data_fully_ready(state):
            heatingSch_hndl = api.exchange.get_actuator_handle(state, "Schedule:Constant", "Schedule Value", "HeatingSch")
            coolingSch_hndl = api.exchange.get_actuator_handle(state, "Schedule:Constant", "Schedule Value", "CoolingSch")
            heatingSP_hndl = api.exchange.get_actuator_handle(state, "Schedule:Constant", "Schedule Value", "hSP")
            coolingSP_hndl = api.exchange.get_actuator_handle(state, "Schedule:Constant", "Schedule Value", "cSP")
            outdoorT_hndl = api.exchange.get_variable_handle(state,'Site Outdoor Air Drybulb Temperature', 'Environment')
            indoorT_hndl = api.exchange.get_variable_handle(state,'Zone Mean Air Temperature', 'Zone1')
            heat_coil_hndl = api.exchange.get_meter_handle(state,'HeatingCoils:EnergyTransfer')
            if -1 in [heatingSch_hndl, coolingSch_hndl, indoorT_hndl, heat_coil_hndl]:
                sys.exit(1)
            handleDone = True
        else:
            return
    
    # exchange information with EnergyPlus
    month = api.exchange.month(state)
    time = api.exchange.current_time(state) 
    
    # read variables
    outdoor_temp = api.exchange.get_variable_value(state, outdoorT_hndl)
    indoor_temp = api.exchange.get_variable_value(state, indoorT_hndl)
    # print(indoor_temp, outdoor_temp)
    
    # read meters
    heating_coils = api.exchange.get_meter_value(state, heat_coil_hndl)
    
    # change setpoints
    heating_SP = 18
    cooling_SP = 27
    
    # actuate
    if month < 4 or month > 10:  # winter
        api.exchange.set_actuator_value(state, heatingSch_hndl, 0)
        api.exchange.set_actuator_value(state, coolingSch_hndl, 1)
        
        if 6<= time < 7:
            heating_SP = 22
        elif 7<= time < 11:
            heating_SP = 19
        elif 11<= time < 17:
            heating_SP = 21
        
        api.exchange.set_actuator_value(state, heatingSP_hndl, heating_SP)
        api.exchange.set_actuator_value(state, coolingSP_hndl, cooling_SP)
        
    else:  # summer
        api.exchange.set_actuator_value(state, heatingSch_hndl, 1)
        api.exchange.set_actuator_value(state, coolingSch_hndl, 0)
        if 6<= time < 7:
            cooling_SP = 23
        elif 7<= time < 10:
            cooling_SP = 25
        elif 10<= time < 11:
            cooling_SP = 23
        elif 11<= time < 17:
            cooling_SP = 25
        
        api.exchange.set_actuator_value(state, heatingSP_hndl, heating_SP)
        api.exchange.set_actuator_value(state, coolingSP_hndl, cooling_SP)
        
    

In [6]:
# initialize EPlus
api = EnergyPlusAPI()

#instance of the simulation
state = api.state_manager.new_state() 

# energyplus model calling point, callback function
api.runtime.callback_begin_system_timestep_before_predictor(state , my_callback_function)

# run EPlus
epwFile = 'ON_OTTAWA-INTL-ONT_716280_19.epw'
idfFile = 'Shoebox.idf'
output_folder = 'out'
# -x short form to run expandobjects for HVACtemplates. see EnergyPlusEssentials.pdf p16
cmd_args = ['-w',epwFile, '-d', output_folder,'-x',idfFile]
api.runtime.run_energyplus(state,cmd_args)

# delete state after simulation to free the memory
api.state_manager.delete_state(state)

print("Handles done: {}".format(handleDone))

Handles done: True


In [7]:
df = pd.read_csv("out/eplusout.csv")
df["timestamps"] = pd.date_range(start="2019/1/1 01:00", end="2020/1/1 00:00", freq='H')
df.set_index("timestamps", inplace=True)

fig, ax = plt.subplots(2,1)
ax[0].plot(df["ZONE1:Zone Thermostat Heating Setpoint Temperature [C](Hourly)"], drawstyle='steps-post')
ax[0].set_xlim([dt.datetime(2019,1,1,0,0,0),dt.datetime(2019,1,2,0,0,0)])
ax[0].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

ax[1].plot(df["ZONE1:Zone Thermostat Cooling Setpoint Temperature [C](Hourly)"], drawstyle='steps-post')
ax[1].set_xlim([dt.datetime(2019,7,1,0,0,0),dt.datetime(2019,7,2,0,0,0)])
ax[1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))


# Demand load control schedule

In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib qt
time_range = pd.date_range(start="04:00", end="20:00", freq='H')

fig, ax = plt.subplots(2,1)
values = [9,9,9, 18, 18, 18, 18, 12,12,12,12 ,12,12,18,18,9,9]
ax[0].plot(time_range, values, drawstyle='steps-post')
ax[0].set_xticks(pd.date_range(start="04:00", end="20:00", freq='2H'), rotation=45)
ax[0].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax[0].set(ylabel="Electricity rate [c\kWh]", title="Heating season")
values = [18,18,22, 19, 19, 19, 19, 21,21,21,21 ,21,21,18,18,18,18]
ax[1].plot(time_range, values, color="r", drawstyle='steps-post')
ax[1].set_xticks(pd.date_range(start="04:00", end="20:00", freq='2H'), rotation=45)
ax[1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax[1].set(ylabel="Heating setpoint [$^\circ$C]")


[Text(0, 0.5, 'Heating setpoint [$^\\circ$C]')]

In [59]:
time_range = pd.date_range(start="04:00", end="20:00", freq='H')

fig, ax = plt.subplots(2,1)
values = [9,9,9, 12, 12, 12, 12, 18,18,18,18 ,18,18,12,12,9,9]
ax[0].plot(time_range, values, drawstyle='steps-post')
ax[0].set_xticks(pd.date_range(start="04:00", end="20:00", freq='2H'), rotation=45)
ax[0].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax[0].set(ylabel="Electricity rate [c\kWh]", title="Cooling season")
values = [27,27,23, 25, 25, 25, 23, 25,25,25,25 ,25,25,27,27,27,27]
ax[1].plot(time_range, values, color="r", drawstyle='steps-post')
ax[1].set_xticks(pd.date_range(start="04:00", end="20:00", freq='2H'), rotation=45)
ax[1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax[1].set(ylabel="Cooling setpoint [$^\circ$C]")


[Text(0, 0.5, 'Cooling setpoint [$^\\circ$C]')]